In [1]:
!pip install datasets
!pip install trl
!pip install language_tool_python
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/si

In [2]:

import torch
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoTokenizer, pipeline

from trl import AutoModelForSeq2SeqLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler


tqdm.pandas()
T5_good_config = PPOConfig(model_name="SummerSigh/T5-Base-Rule-Of-Thumb", learning_rate=1e-5, batch_size=4)
T5_evil_config = PPOConfig(model_name="SummerSigh/t5-ROTLabel-to-Prompt", learning_rate=1e-5, batch_size=4)

In [3]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from datetime import datetime
import logging

import pandas as pd
from sklearn.model_selection import train_test_split


dataset = load_dataset("allenai/prosocial-dialog", split='train')
prosocial_df = dataset.to_pandas()
prosocial_df = prosocial_df[["context","rots","safety_label", "episode_done", "response"]]

episode_done = False  
data_inputs = []
data_optputs = []
current_convo = ""
for i, data in prosocial_df.iterrows():
    user = "<user>" + data["context"]
    assistant_reason = "<reason>" + data["rots"][0] + " " + data["safety_label"]
    assistant_response = "<assistant>" + data["response"]
    if current_convo == "":
        data_inputs.append(user)
        data_optputs.append(assistant_reason)
    else: 
        data_inputs.append(current_convo + user)
        data_optputs.append(assistant_reason)
    if data["episode_done"] == False: 
        current_convo = current_convo + user + assistant_response
    else:
        current_convo = ""
final_df_good = pd.DataFrame({"input_text": data_inputs, "target_text": data_optputs})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/allenai___json/allenai--prosocial-dialog-ebbad39ca08b6d44/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


In [4]:
# We then define the arguments to pass to the sentiment analysis pipeline.
# We set `return_all_scores` to True to get the sentiment score for each token.
tokenizer = AutoTokenizer.from_pretrained(T5_evil_config.model_name)

def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["input_text"]) + [tokenizer.eos_token_id]
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample
set_seed(T5_good_config.seed)

In [5]:
final_df_evil = pd.DataFrame({"target_text": data_inputs, "input_text": data_optputs})
dataset_evil = Dataset.from_pandas(final_df_evil)
dataset_evil = dataset_evil.map(tokenize, batched=False)
dataset_evil.set_format(type="torch")

Map:   0%|          | 0/120236 [00:00<?, ? examples/s]

In [6]:
# Now let's build the model, the reference model, and the tokenizer.
model_good = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(T5_good_config.model_name)
ref_model_good = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(T5_good_config.model_name)
model_Evil = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(T5_evil_config.model_name)
ref_model_evil = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(T5_evil_config.model_name)
tokenizer = AutoTokenizer.from_pretrained(T5_good_config.model_name)
def collater(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

query = tokenizer("I really liked this movie because", return_tensors="pt")["input_ids"]


# We then build the PPOTrainer, passing the model, the reference model, the tokenizer
ppo_trainer_good = PPOTrainer(T5_good_config, model_good, ref_model_good, tokenizer, data_collator=collater)
ppo_trainer_evil = PPOTrainer(T5_evil_config, model_Evil, ref_model_evil, tokenizer, dataset=dataset_evil, data_collator=collater)


/usr/local/lib/python3.9/dist-packages/trl/trainer/ppo_trainer.py:213: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [8]:
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpn_titozq.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


In [10]:
from transformers import pipeline
pipe = pipeline("text-classification", model = "madhurjindal/autonlp-Gibberish-Detector-492513457")

In [12]:
import re
from sentence_transformers import SentenceTransformer, util
import torch

text_sim_checker = SentenceTransformer('all-MiniLM-L6-v2')

def compute_sim(text1,text2):
    text1 = re.sub(r'__.*?__', '', text1)
    text2 = re.sub(r'__.*?__', '', text2)
    embeddings1 = text_sim_checker.encode(text1, convert_to_tensor=True)
    embeddings2 = text_sim_checker.encode(text2, convert_to_tensor=True)
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    return cosine_scores

def compare_verdicts(text1, text2):
    """
    Compares the final verdicts of two input texts and returns a similarity score based on the severity levels of the verdicts.
    """
    # Define a dictionary mapping verdicts to severity levels
    severity_levels = {
        'casual': 1,
        'possibly_needs_caution': 2,
        'probably_needs_caution': 3,
        'needs_caution': 4,
        'needs_intervention': 5
    }
    
    # Extract the final verdicts from each input text using regular expressions
    pattern = re.compile(r'__(.*?)__')
    verdicts1 = pattern.findall(text1)
    verdicts2 = pattern.findall(text2)

    
    # Compute the maximum severity level of the verdicts in each input text
    try:
      max_severity1 = max([severity_levels[v] for v in verdicts1])
      max_severity2 = max([severity_levels[v] for v in verdicts2])
      # Compute the difference in severity levels between the two input texts
      severity_diff = abs(max_severity1 - max_severity2)
      
      # Return a similarity score based on the severity difference
      if severity_diff == 0:
          reward = 1.0
      elif severity_diff == 1:
          reward = 0.75
      elif severity_diff == 2:
          reward = 0.25
      else:
          reward = 0.0
    except:
      reward = 0.0
    return reward


def compute_reward(text_good, text_evil, text_rot):

  reward_good = compute_sim(text_good, text_rot)
  reward_verdict = compare_verdicts(text_good, text_rot)
  reward_good += reward_verdict
  reward_bad = compute_sim(text_evil, text_rot)
  reward_bad += abs(reward_verdict - 1)
  reward_good = torch.tensor(reward_good) 
  reward_bad = torch.tensor(reward_bad) 
  return reward_good, reward_bad

In [ ]:
# We then define the arguments to pass to the `generate` function. These arguments
# are passed to the `generate` function of the PPOTrainer, which is a wrapper around
# the `generate` function of the trained model.
generation_kwargs_good = {"top_k": 50, "top_p": 1.0, "do_sample": True, "temperature": 0.9}
generation_kwargs_evil = {"top_k": 50, "top_p": 1.0, "do_sample": True}
generation_kwargs_good["max_new_tokens"] = 500
generation_kwargs_evil["max_new_tokens"] = 200
generation_kwargs_evil["min_new_tokens"] = 20


with open('prompts.txt', 'a') as f:
  for epoch, batch in tqdm(enumerate(ppo_trainer_evil.dataloader)):
    query_tensors = batch["input_ids"]
    # Get response from gpt2
    response_tensors_bad = []
    for query in query_tensors:
      response = ppo_trainer_evil.generate(query, **generation_kwargs_evil)
      response_tensors_bad.append(response.squeeze())
    responce_good_tensors = []
    for responce in response_tensors_bad:
      responce_good = ppo_trainer_good.generate(responce, **generation_kwargs_good)
      responce_good_tensors.append(responce_good.squeeze())
    text_new =[tokenizer.decode(r[1:].squeeze(), add_special_tokens = False) for r in responce_good_tensors]
    text_new_evil =[tokenizer.decode(r[1:].squeeze(), add_special_tokens = False) for r in response_tensors_bad]
    text_old = batch["query"]
    rewards_good = []
    rewards_bad = []
    for i in range(len(text_old)):
      good_reward, bad_reward = compute_reward(text_new[i],text_new_evil[i],text_old[i])
      f.write("[" + text_old[i] + "," + text_new_evil[i] + "," + text_new[i] + "," + str(bad_reward) + "," + str(good_reward) + "],")
      rewards_good.append(good_reward.squeeze())
      rewards_bad.append(bad_reward.squeeze())
    stats = ppo_trainer_evil.step(query_tensors, response_tensors_bad, rewards_good)
    ppo_trainer_evil.log_stats(stats, batch, rewards_bad)
    stats = ppo_trainer_good.step(query_tensors, response_tensors_bad, rewards_bad)
    ppo_trainer_good.log_stats(stats, batch, rewards_good)



0it [00:00, ?it/s]<ipython-input-12-de5889a1d50c>:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reward_good = torch.tensor(reward_good)
<ipython-input-12-de5889a1d50c>:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reward_bad = torch.tensor(reward_bad)
30it [05:38, 11.18s/it]